In [27]:
import pandas as pd
import pulp as lp
import numpy as np
from operator import iadd
from functools import reduce
from typing import Sequence, Any, DefaultDict, List

In [28]:
def create_prob(prob_name: str, sense: int) -> lp.LpProblem:
    return lp.LpProblem(prob_name, sense)


def add_obj_fn(lp_prob: lp.LpProblem, dvar: lp.LpAffineExpression) -> lp.LpProblem:
    return iadd(lp_prob, dvar)


def add_constraint(lp_prob: lp.LpProblem, constrs: Sequence[lp.LpConstraint]) -> lp.LpProblem:
   return reduce(iadd, constrs, lp_prob)


def head(x: Sequence) -> Any:
    return x[0]

def to_str(indnum, activity) -> str:
    return f'{indnum} - {activity}'

In [29]:
df = pd.read_csv('data/dataset.csv')
ql_df = pd.read_csv('data/indv_tql.csv')

df.drop('Unnamed: 0', axis=1, inplace=True) # Drop Unnamed column
ql_df.drop('Unnamed: 0', axis=1, inplace=True) # Drop Unnamed column

In [30]:
#df.head()
#ql_df.head()

In [31]:
indv1_df = df.drop('Group', axis=1).loc[df['Indnum'] == 1]
indv1_tql = ql_df.loc[ql_df['Indnum'] == 1]

In [32]:
#indv1_df.head()
#indv1_tql

In [33]:
gp_model = create_prob('Wells Fargo Challenge', lp.LpMinimize)

## Decision Variables

In [34]:
indv_num = indv1_df.Indnum.unique()
actv_names = indv1_df.Activity.unique()

sources = np.array([
  "solar_powered_water_heater",
  "gas_water_heater",
  "electric_water_heater_peak_hour",
  "electric_water_heater_off_peak",
  "gas",
  "natural_gas",
  "hybrid",
  "electric_peak_hours",
  "electric_off_peak_hours",
  "jetfuel"
])

In [35]:
actv_indexes = [
    (indv, activity)
    for indv in indv_num
    for activity in actv_names
]

TrCF_ij = lp.LpVariable.dicts('TrCF_ij', actv_indexes, lowBound=0)

In [36]:
actv_indexes = [
    (f'{indv} - {activity}', source)
    for indv in indv_num
    for activity in actv_names
    for source in sources
]

CF_ij = lp.LpVariable.dicts('CF_ij', actv_indexes, lowBound=0)

In [37]:
consumption_indexes = [
    (indv, activity)
    for indv in indv_num
    for activity in actv_names
]

C_ij = lp.LpVariable.dicts('C_ij', consumption_indexes, lowBound=0)

In [38]:
qual_life_indexes = [
    (indv, activity)
    for indv in indv_num
    for activity in actv_names
]

TrQL_ij = lp.LpVariable.dicts('TrQL_ij', qual_life_indexes, lowBound=0)

## Objective Function

In [39]:
obj_fn = lp.lpSum([TrCF_ij[(indv, activity)] for indv in indv_num for activity in actv_names ])

obj_fn

1*TrCF_ij_(1,_'Household_heating_<_70F') + 1*TrCF_ij_(1,_'Household_heating_=__70F') + 1*TrCF_ij_(1,_'Small_kitchen_appliance_in_the_home') + 1*TrCF_ij_(1,_'TV_computer_use') + 1*TrCF_ij_(1,_'Use_of_air_conditioner') + 1*TrCF_ij_(1,_'Use_of_heat_pump') + 1*TrCF_ij_(1,_'air_travel___large_plane') + 1*TrCF_ij_(1,_'air_travel___small__plane_(<50_seats)') + 1*TrCF_ij_(1,_'bags_of_compost_deposited_(negative_CF)') + 1*TrCF_ij_(1,_'bags_of_garbage_disposed') + 1*TrCF_ij_(1,_'bags_of_recycling_deposited_(negative_CF)') + 1*TrCF_ij_(1,_'bath') + 1*TrCF_ij_(1,_'car_trips___2__people_with_multiple_end_points') + 1*TrCF_ij_(1,_'car_trips___driver_and_self') + 1*TrCF_ij_(1,_'car_trips__self_only') + 1*TrCF_ij_(1,_'hazardous_or_electric_items_disposed') + 1*TrCF_ij_(1,_'large_items_disposed') + 1*TrCF_ij_(1,_'shower___long_(__3_min)') + 1*TrCF_ij_(1,_'shower___short') + 1*TrCF_ij_(1,_'trips_using_public_ground_transportation') + 1*TrCF_ij_(1,_'use_of__oven') + 1*TrCF_ij_(1,_'use_of_clothes_dryer') 

## Constraints

In [40]:
consup_conds = []

for indv in indv_num:
    for activity in actv_names:
            consumption: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'Consumption'].values
            condition = C_ij[(indv, activity)] <= head(consumption)
            
            consup_conds.append(condition)
#consup_conds

In [41]:
source_cf_conds = []

for indv in indv_num:
    for activity in actv_names:
        for source in sources:
            source_cf: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, source].values
            condition =  CF_ij[(to_str(indv, activity), source)] <= head(source_cf)
            
            source_cf_conds.append(condition)
            
#source_cf_conds

In [42]:
activity_cf_conds = []

for indv in indv_num:
    for activity in actv_names:
        activity_sum_cf: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'TCF'].values
        activity_source_cf = []
        
        for source in sources:
            activity_source_cf.append(CF_ij[(to_str(indv, activity), source)])
            
        activity_cf_conds.append(lp.lpSum(activity_source_cf) <= activity_sum_cf)

In [43]:
tr_cf_conds = []

for indv in indv_num:
    for activity in actv_names:
        activity_source_cf = []
        activity_sum_cf: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'TCF'].values
        for source in sources:
            activity_source_cf.append(CF_ij[(to_str(indv, activity), source)])
            
        tr_cf_conds.append(TrCF_ij[(indv, activity)] == activity_sum_cf * C_ij[(indv, activity)])

tr_cf_conds

[-0.000436*C_ij_(1,_'Household_heating_=__70F') + 1*TrCF_ij_(1,_'Household_heating_=__70F') + 0.0 = 0,
 -0.000872*C_ij_(1,_'Household_heating_<_70F') + 1*TrCF_ij_(1,_'Household_heating_<_70F') + 0.0 = 0,
 1*TrCF_ij_(1,_'Use_of_heat_pump') + 0 = 0,
 1*TrCF_ij_(1,_'Use_of_air_conditioner') + 0 = 0,
 1*TrCF_ij_(1,_'shower___short') + 0 = 0,
 1*TrCF_ij_(1,_'shower___long_(__3_min)') + 0 = 0,
 1*TrCF_ij_(1,_'bath') + 0 = 0,
 -4.2e-05*C_ij_(1,_'wash_up') + 1*TrCF_ij_(1,_'wash_up') + 0.0 = 0,
 1*TrCF_ij_(1,_'use_of_dishwasher') + 0 = 0,
 1*TrCF_ij_(1,_'use_of_clothes_washer') + 0 = 0,
 1*TrCF_ij_(1,_'use_of_clothes_dryer') + 0 = 0,
 1*TrCF_ij_(1,_'use_of_cooking_range') + 0 = 0,
 1*TrCF_ij_(1,_'use_of__oven') + 0 = 0,
 1*TrCF_ij_(1,_'use_of_self_clean_feature_of_electric_oven') + 0 = 0,
 1*TrCF_ij_(1,_'Small_kitchen_appliance_in_the_home') + 0 = 0,
 1*TrCF_ij_(1,_'TV_computer_use') + 0 = 0,
 1*TrCF_ij_(1,_'air_travel___large_plane') + 0 = 0,
 1*TrCF_ij_(1,_'air_travel___small__plane_(<50_seat

In [44]:
tr_ql_conds = []

for indv in indv_num:
    indv_tr_ql = []
    for activity in actv_names:
        QL: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'Quality_of_Life_Importance__1_10'].values
        indv_tr_ql.append(C_ij[(indv, activity)] * head(QL))
        
    tr_ql_conds.append(TrQL_ij[(indv, activity)] == lp.lpSum(indv_tr_ql))
    
#tr_ql_conds

In [45]:
indv_tql_conds = []

for indv in indv_num:
    for activity in actv_names:
            indv_tql: np.ndarray = indv1_tql.loc[indv1_tql['Indnum'] == 1, 'TQL'].values
            indv_tql_conds.append(TrQL_ij[(indv, activity)] >= indv_tql)
#indv_tql_conds

In [46]:
gp_model = add_obj_fn(gp_model, obj_fn)

gp_model = add_constraint(gp_model, consup_conds)

gp_model = add_constraint(gp_model, source_cf_conds)

gp_model = add_constraint(gp_model, activity_cf_conds)

gp_model = add_constraint(gp_model, tr_cf_conds)

gp_model = add_constraint(gp_model, tr_ql_conds)

gp_model = add_constraint(gp_model, indv_tql_conds)

In [47]:
gp_model.solve()

1

In [48]:
lp.LpStatus[gp_model.status]

'Optimal'

In [49]:
lp.value(gp_model.objective)

0.0

In [50]:
for indv in indv_num:
    for activity in actv_names:
        for source in sources:
            print(lp.value(CF_ij[(to_str(indv, activity), source)]))
            
            

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
